In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# =============================================================================
# 1. IMPORTACIONES
# =============================================================================
import pandas as pd
import numpy as np
import pickle
from scipy.stats import randint, uniform

# Preprocesamiento y Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# ¡Importaciones clave para el transformador personalizado!
from sklearn.base import BaseEstimator, TransformerMixin

# Modelo
from xgboost import XGBClassifier

# Ignorar advertencias
import warnings
warnings.filterwarnings('ignore')


# =============================================================================
# 2. TRANSFORMADOR DE LIMPIEZA PERSONALIZADO
# (Este reemplaza la lógica de DataManager.load_and_clean_data)
# =============================================================================

class DataCleanerTransformer(BaseEstimator, TransformerMixin):
    """
    Este transformador realiza la limpieza de datos crudos que NO
    elimina filas, como limpiar strings o eliminar columnas.
    """
    def __init__(self):
        self.obj_cols = []
        self.all_null_cols = []
        print("DataCleanerTransformer inicializado (sin drop_duplicates).")

    def fit(self, X, y=None):
        """
        Aprende qué columnas son 'object' y cuáles están 100% nulas.
        """
        print("[Cleaner Fit] Aprendiendo estructura de datos...")
        df_temp = X.replace(r"^\s*$", np.nan, regex=True).replace({"NA": np.nan, "N/A": np.nan, "na": np.nan, "NaN": np.nan})

        # 1. Aprender qué columnas son de tipo 'object' para limpiarlas
        self.obj_cols = df_temp.select_dtypes(include=["object"]).columns.tolist()

        # 2. Aprender qué columnas están 100% nulas *en el training set*
        self.all_null_cols = [c for c in df_temp.columns if df_temp[c].isna().all()]

        print(f"[Cleaner Fit] Columnas 'object' a limpiar: {self.obj_cols}")
        print(f"[Cleaner Fit] Columnas 100% nulas a eliminar: {self.all_null_cols}")
        return self

    def transform(self, X):
        """
        Aplica la limpieza a cualquier dato (train o test/new).
        """
        print("[Cleaner Transform] Aplicando limpieza de datos crudos...")
        df = X.copy()

        # 1. Estandarizar nulos (basado en DataManager)
        df = df.replace(r"^\s*$", np.nan, regex=True)
        df = df.replace({"NA": np.nan, "N/A": np.nan, "na": np.nan, "NaN": np.nan})

        # 2. Recortar strings (usando las columnas aprendidas en fit)
        for c in self.obj_cols:
            if c in df.columns:
                df[c] = df[c].astype(str).str.strip()

        # 3. Eliminar columnas 100% nulas (aprendidas en fit)
        if self.all_null_cols:
            df = df.drop(columns=self.all_null_cols, errors='ignore')

        # 4. (ELIMINADO) df = df.drop_duplicates()
        # ¡Ya no eliminamos duplicados aquí!

        print("[Cleaner Transform] Limpieza completada.")
        return df

# =============================================================================
# 3. CLASE DE FÁBRICA DE PIPELINES
# (Modificada para construir el pipeline de 3 pasos)
# =============================================================================

class ModelPipelineFactory:
    """
    Construye y entrena pipelines de modelo.
    """
    def __init__(self):
        print("ModelPipelineFactory inicializada.")

    def create_preprocessor(self, X_train_cleaned):
        """
        Crea un preprocesador (ColumnTransformer) basado en los tipos de
        columnas de un DataFrame YA LIMPIO.

        :param X_train_cleaned: DataFrame de entrenamiento DESPUÉS de la limpieza.
        :return: Objeto ColumnTransformer.
        """
        # Identificar tipos de columnas desde X_train_cleaned
        numeric_features = X_train_cleaned.select_dtypes(include=np.number).columns
        categorical_features = X_train_cleaned.select_dtypes(
            include=['object', 'category']
        ).columns

        print(f"Preprocesador: {len(numeric_features)} features numéricas y {len(categorical_features)} categóricas.")

        # Pipeline para variables numéricas
        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value=-20)),
            ('scaler', RobustScaler())
        ])

        # Pipeline para variables categóricas
        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
            ('onehot', OneHotEncoder(
                handle_unknown='ignore', sparse_output=False, drop='first'
            ))
        ])

        # Crear el ColumnTransformer
        preprocessor = ColumnTransformer(transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ], remainder='passthrough')

        return preprocessor

    def create_and_tune_model(self, X_train_raw, y_train, preprocessor, classifier, param_grid, search_options):
        """
        Crea un pipeline completo de 3 PASOS (Cleaner -> Preprocessor -> Classifier)
        y lo entrena usando RandomizedSearchCV.

        :param X_train_raw: Características de entrenamiento CRUDAS.
        :param y_train: Objetivo de entrenamiento (codificado).
        :param preprocessor: El ColumnTransformer ya definido.
        :param classifier: El objeto clasificador (ej. XGBClassifier()).
        :param param_grid: Diccionario de hiperparámetros.
        :param search_options: Opciones para RandomizedSearchCV.
        :return: Objeto RandomizedSearchCV ajustado.
        """

        # 2. Crear el pipeline principal de 3 PASOS
        model_pipeline = Pipeline(steps=[
            ('cleaner', DataCleanerTransformer()),  # PASO 1: Limpieza
            ('preprocessor', preprocessor),         # PASO 2: Preprocesamiento
            ('classifier', classifier)              # PASO 3: Modelo
        ])

        # 3. Configurar y ejecutar la búsqueda aleatoria
        random_search = RandomizedSearchCV(
            model_pipeline,
            param_distributions=param_grid,
            **search_options
        )

        model_name = classifier.__class__.__name__
        print(f"Iniciando RandomizedSearchCV para el Pipeline completo ({model_name})...")
        # ¡Entrenamos el pipeline completo con los datos CRUDOS!
        random_search.fit(X_train_raw, y_train)
        print(f"Búsqueda completada para {model_name}.")

        return random_search


# =============================================================================
# 4. BLOQUE PRINCIPAL DE EJECUCIÓN (Entrenamiento y Guardado)
# =============================================================================

if __name__ == "__main__":

    # --- 1. Configuración Inicial ---
    SRC_PATH = '/content/drive/MyDrive/Pregrado - Posgrado - Trabajo/Maestría - Inteligencia Artificial Aplicada/11. MLOps/1. Primera etapa de proyecto/Modelado/obesity_estimation_original.csv'
    TARGET = 'NObeyesdad'
    PICKLE_FILE = 'xgboost_model_artifacts.pkl'

    print("Iniciando Proceso de ENTRENAMIENTO y GUARDADO (Raw-to-Prediction)...")
    print("="*80)

    # --- 2. Carga y LIMPIEZA INICIAL de Datos CRUDOS ---
    try:
        df_raw = pd.read_csv(SRC_PATH, sep=None, engine="python", encoding="utf-8")
        print("Datos crudos cargados exitosamente.")
    except Exception as e:
        df_raw = pd.read_csv(SRC_PATH, sep=';', engine="python", encoding="utf-8")
        print("Datos crudos cargados con separador ';'.")

    # --- ¡SOLUCIÓN AQUÍ! ---
    # Aplicamos la eliminación de duplicados al DataFrame COMPLETO
    # antes de hacer cualquier otra cosa.
    duplicates_before = df_raw.duplicated().sum()
    if duplicates_before > 0:
        df_raw = df_raw.drop_duplicates()
        print(f"Se eliminaron {duplicates_before} duplicados del DataFrame crudo.")
    else:
        print("No se encontraron duplicados en los datos crudos.")
    # --- Fin de la solución ---

    # Ahora X_raw e y_raw ya no tienen duplicados
    X_raw = df_raw.drop(TARGET, axis=1)
    y_raw = df_raw[TARGET]

    # Dividimos los datos (que ya están libres de duplicados)
    X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
        X_raw, y_raw, test_size=0.2, random_state=42, stratify=y_raw
    )
    print(f"Datos (sin duplicados) divididos: {X_train_raw.shape[0]} para entreno, {X_test_raw.shape[0]} para test.")

    # --- 3. Codificación de 'y' (Variable Objetivo) ---
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train_raw)
    y_test_encoded = label_encoder.transform(y_test_raw)
    print(f"Variable objetivo codificada. Clases: {label_encoder.classes_}")

    # --- 4. Creación del Preprocesador (Paso de Inspección) ---
    # (Esto sigue igual, pero el 'cleaner_inspector' ya no tiene drop_duplicates)
    print("\nInspeccionando datos de entreno para configurar el preprocesador...")
    cleaner_inspector = DataCleanerTransformer()
    X_train_cleaned_for_inspection = cleaner_inspector.fit_transform(X_train_raw)

    factory = ModelPipelineFactory()
    preprocessor = factory.create_preprocessor(X_train_cleaned_for_inspection)
    print("Preprocesador (ColumnTransformer) creado exitosamente.")

    # --- 5. Configuración del Modelo XGBoost (Sigue igual) ---
    xgb_config = {
        "classifier": XGBClassifier(
            objective='multi:softmax', eval_metric='mlogloss',
            use_label_encoder=False, random_state=42
        ),
        "param_grid": {
            'classifier__n_estimators': randint(100, 500),
            'classifier__max_depth': randint(3, 10),
            'classifier__learning_rate': uniform(0.01, 0.3),
        }
    }
    SEARCH_OPTIONS = {
        'n_iter': 50, 'cv': 5, 'scoring': 'accuracy',
        'verbose': 1, 'random_state': 42, 'n_jobs': -1
    }

    # --- 6. Entrenamiento del Pipeline Completo ---
    # (Sigue igual. Ahora X_train_raw y y_train_encoded tienen el mismo
    # número de filas, y el pipeline no cambiará ese número).
    print("\n" + "="*80)
    print(" INICIANDO ENTRENAMIENTO DEL PIPELINE COMPLETO (RAW-TO-PREDICTION)")
    print("="*80)

    search_cv = factory.create_and_tune_model(
        X_train_raw,
        y_train_encoded,
        preprocessor,
        xgb_config["classifier"],
        xgb_config["param_grid"],
        SEARCH_OPTIONS
    )

    best_model = search_cv.best_estimator_

    # --- 7. Resultados y Evaluación (Sigue igual) ---
    print("\n" + "--- RESULTADOS FINALES ---")
    print(f"Mejor puntaje CV (Accuracy): {search_cv.best_score_:.4f}")
    accuracy_test = best_model.score(X_test_raw, y_test_encoded)
    print(f"Precisión final en Test (con datos crudos): {accuracy_test:.4f}\n")

    # --- 8. Guardado de Artefactos (Sigue igual) ---
    print("="*80)
    print(f"Guardando artefactos del modelo en: {PICKLE_FILE}")

    model_artifacts = {
        'model': best_model,
        'label_encoder': label_encoder
    }

    try:
        with open(PICKLE_FILE, 'wb') as f:
            pickle.dump(model_artifacts, f)
        print("¡Artefactos guardados exitosamente!")
    except Exception as e:
        print(f"Error al guardar el archivo pickle: {e}")

    print("="*80)
    print("Proceso de Entrenamiento y Guardado Finalizado.")
    print("="*80)

Iniciando Proceso de ENTRENAMIENTO y GUARDADO (Raw-to-Prediction)...
Datos crudos cargados exitosamente.
Se eliminaron 24 duplicados del DataFrame crudo.
Datos (sin duplicados) divididos: 1669 para entreno, 418 para test.
Variable objetivo codificada. Clases: ['Insufficient_Weight' 'Normal_Weight' 'Obesity_Type_I' 'Obesity_Type_II'
 'Obesity_Type_III' 'Overweight_Level_I' 'Overweight_Level_II']

Inspeccionando datos de entreno para configurar el preprocesador...
DataCleanerTransformer inicializado (sin drop_duplicates).
[Cleaner Fit] Aprendiendo estructura de datos...
[Cleaner Fit] Columnas 'object' a limpiar: ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
[Cleaner Fit] Columnas 100% nulas a eliminar: []
[Cleaner Transform] Aplicando limpieza de datos crudos...
[Cleaner Transform] Limpieza completada.
ModelPipelineFactory inicializada.
Preprocesador: 8 features numéricas y 8 categóricas.
Preprocesador (ColumnTransformer) creado exitosamen

# Model usage

In [10]:
import pickle

with open('xgboost_model_artifacts.pkl', 'rb') as f:
    artifacts = pickle.load(f)

model = artifacts['model']
label_encoder = artifacts['label_encoder']

In [14]:
df_raw = pd.read_csv(SRC_PATH, sep=None, engine="python", encoding="utf-8")

# new_data_raw es un DataFrame con los datos crudos
new_data_cleaned = df_raw # (Aquí aplicas la lógica de DataManager)

# 1. Predecir las clases numéricas
pred_numeric = model.predict(new_data_cleaned)

# 2. Predecir las probabilidades (si las necesitas)
pred_proba = model.predict_proba(new_data_cleaned)

# 3. Decodificar a etiquetas legibles
pred_labels = label_encoder.inverse_transform(pred_numeric)

print(pred_labels)

[Cleaner Transform] Aplicando limpieza de datos crudos...
[Cleaner Transform] Limpieza completada.
[Cleaner Transform] Aplicando limpieza de datos crudos...
[Cleaner Transform] Limpieza completada.
['Normal_Weight' 'Normal_Weight' 'Normal_Weight' ... 'Obesity_Type_III'
 'Obesity_Type_III' 'Obesity_Type_III']
